In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [3]:
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

In [4]:
ds_train = ds_train.map(
    normalize, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [5]:
ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# 1

In [6]:
inp = tf.keras.layers.Input((28, 28, 1))

l = tf.keras.layers.Conv2D(2, 5, 1, padding='same')(inp)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)
l2 = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)
l = tf.keras.layers.MaxPool2D()(l)

l2 = tf.keras.layers.Add()([l2,l])

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l2)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)

l2 = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)
l = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Add()([l, l2])

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)

l2 = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)
l = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Add()([l, l2])

l = tf.keras.layers.Conv2D(2, 3, 1, padding='same')(l)
l = tf.keras.layers.ReLU()(l)
l = tf.keras.layers.BatchNormalization()(l)

l = tf.keras.layers.Flatten()(l)

outp = tf.keras.layers.Dense(10, activation="softmax")(l)

model = tf.keras.Model(inputs=inp, outputs=outp)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 2)    52          ['input_1[0][0]']                
                                                                                                  
 re_lu (ReLU)                   (None, 28, 28, 2)    0           ['conv2d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 2)   8           ['re_lu[0][0]']                  
 alization)                                                                                   

# 2

In [50]:
class ResBlock(tf.keras.layers.Layer):
    def __init__(self, channel, start_kernel_size=5):
        super(ResBlock, self).__init__()
        self.c1 = tf.keras.layers.Conv2D(channel, start_kernel_size, 1, padding='same', activation="relu")
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.c2 = tf.keras.layers.Conv2D(channel, 3, 1, padding='same', activation="relu")
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.add = tf.keras.layers.Add()
        
    def call(self, inp):
        l = self.c1(inp)
        l2 = self.bn1(l)

        l = self.c2(l)
        l = self.bn2(l)
        
        a = self.add([l, l2])
        
        return a

inp = tf.keras.layers.Input((28, 28, 1))

l = ResBlock(2,3)(inp)
l = tf.keras.layers.MaxPool2D()(l)
l = ResBlock(2,3)(l)
l = tf.keras.layers.MaxPool2D()(l)
l = ResBlock(2,3)(l)
l = tf.keras.layers.MaxPool2D()(l)
l = ResBlock(2,3)(l)

l = tf.keras.layers.Conv2D(10,1,1, padding="same", activation="relu")(l)
l = tf.keras.layers.GlobalAveragePooling2D()(l)

outp = tf.keras.layers.Softmax()(l)

model = tf.keras.Model(inputs=inp, outputs=outp)

model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 res_block_104 (ResBlock)    (None, 28, 28, 2)         74        
                                                                 
 max_pooling2d_84 (MaxPoolin  (None, 14, 14, 2)        0         
 g2D)                                                            
                                                                 
 res_block_105 (ResBlock)    (None, 14, 14, 2)         92        
                                                                 
 max_pooling2d_85 (MaxPoolin  (None, 7, 7, 2)          0         
 g2D)                                                            
                                                                 
 res_block_106 (ResBlock)    (None, 7, 7, 2)           92 

# 3

In [75]:
class ResBlock(tf.keras.layers.Layer):
    def __init__(self, channel, start_kernel_size=5):
        super(ResBlock, self).__init__()
        self.c1 = tf.keras.layers.Conv2D(channel, start_kernel_size, 1, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.c2 = tf.keras.layers.Conv2D(channel, 3, 1, padding='same', activation="relu")
        
        self.add = tf.keras.layers.Add()
        
    def call(self, inp):
        l = self.c1(inp)
        l = self.bn1(l)
        l1 = self.relu(l)

        l = self.c2(l1)
        
        a = self.add([l, l1])
        
        return a

inp = tf.keras.layers.Input((28, 28, 1))

l = ResBlock(8,5)(inp)
l = tf.keras.layers.MaxPool2D()(l)
l = ResBlock(16,5)(l)
l = tf.keras.layers.MaxPool2D()(l)
l = ResBlock(4,3)(l)
l = tf.keras.layers.MaxPool2D()(l)

l = tf.keras.layers.Flatten()(l)

outp = tf.keras.layers.Dense(10, activation="softmax")(l)

model = tf.keras.Model(inputs=inp, outputs=outp)

model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 res_block_96 (ResBlock)     (None, 28, 28, 8)         824       
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 14, 14, 8)        0         
 g2D)                                                            
                                                                 
 res_block_97 (ResBlock)     (None, 14, 14, 16)        5600      
                                                                 
 max_pooling2d_76 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                                            
                                                                 
 res_block_98 (ResBlock)     (None, 7, 7, 4)           744

In [76]:
model.compile(
    tf.keras.optimizers.Adam(5e-3), 
    'sparse_categorical_crossentropy',
    metrics=["acc"]
)

In [77]:
model.fit(ds_train, epochs=10)

Epoch 1/10
469/469 [==============================] - 14s 28ms/step - loss: 0.2805 - acc: 0.9180
Epoch 2/10
469/469 [==============================] - 14s 29ms/step - loss: 0.0643 - acc: 0.9794
Epoch 3/10
469/469 [==============================] - 13s 29ms/step - loss: 0.0510 - acc: 0.9838
Epoch 4/10
469/469 [==============================] - 14s 29ms/step - loss: 0.0441 - acc: 0.9865
Epoch 5/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0364 - acc: 0.9890
Epoch 6/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0365 - acc: 0.9880
Epoch 7/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0303 - acc: 0.9904
Epoch 8/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0284 - acc: 0.9911
Epoch 9/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0289 - acc: 0.9907
Epoch 10/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0267 - acc: 0.9911


In [68]:
model.evaluate(ds_test)

79/79 [==============================] - 1s 7ms/step - loss: 0.0610 - acc: 0.9801


[0.061046067625284195, 0.9800999760627747]